### Tahap 1: Pemuatan Data dan Inisialisasi

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Saat ini TensorFlow mencoba mengalokasikan semua memori GPU.
    # Kode ini membatasi pertumbuhan memori agar lebih efisien.
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print(f"Berhasil mendeteksi {len(gpus)} GPU: {gpus}")
  except RuntimeError as e:
    # Terjadi error jika memory growth sudah di-set sebelumnya
    print(e)
else:
  print("Peringatan: Tidak ada GPU yang terdeteksi. Model akan dilatih menggunakan CPU (jauh lebih lambat).")



Peringatan: Tidak ada GPU yang terdeteksi. Model akan dilatih menggunakan CPU (jauh lebih lambat).


In [3]:
# --- Atur Path (Gunakan kode dari notebook 00 yang sudah direvisi) ---
# BASE_DIR, PATH_SPLIT_DATA, PATH_PREDICTIONS, dll.

# --- Muat Data ---
path_split_data = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\split_data' # Path ke folder split_data
train_df = pd.read_parquet(os.path.join(path_split_data, 'train_set.parquet'))
test_df = pd.read_parquet(os.path.join(path_split_data, 'test_set.parquet'))

# --- Definisikan Fitur dan Target ---
TARGET = 'konsumsi_energi'
# Hapus fitur non-numerik atau yang tidak relevan untuk LSTM (seperti 'apakah_akhir_pekan')
FEATURES = [col for col in train_df.columns if col not in ['timestamp', 'meter_id', TARGET]]

print("Data Latih dan Uji berhasil dimuat.")
print("Fitur yang akan digunakan:", FEATURES)


Data Latih dan Uji berhasil dimuat.
Fitur yang akan digunakan: ['is_kelas', 'is_kantor', 'is_penelitian', 'avg_temp_previous_hour', 'jam', 'hari_minggu', 'hari_bulan', 'minggu_tahun', 'bulan', 'tahun', 'apakah_akhir_pekan', 'apakah_jam_kerja', 'konsumsi_lag_1_jam', 'konsumsi_lag_24_jam']


### Tahap 2: Pra-pemrosesan Data

In [4]:
# --- Konfigurasi LSTM ---
N_PAST = 24 # Jumlah jam masa lalu yang digunakan untuk prediksi (contoh)
N_FUTURE = 1 # Memprediksi 1 jam ke depan

X_train, y_train = [], []
X_test, y_test = [], []
scalers = {} # Dictionary untuk menyimpan scaler untuk setiap gedung

# --- Proses Data Latih ---
print("\nMemproses Data Latih...")
for meter_id, group in train_df.groupby('meter_id'):
    # 1. Scaling: Fit dan transform HANYA pada data latih gedung ini
    scaler = MinMaxScaler()
    group_scaled = scaler.fit_transform(group[FEATURES + [TARGET]])
    scalers[meter_id] = scaler # Simpan scaler untuk digunakan pada data uji nanti

    # 2. Buat Sekuens
    for i in range(N_PAST, len(group_scaled) - N_FUTURE + 1):
        X_train.append(group_scaled[i - N_PAST:i, 0:len(FEATURES)])
        y_train.append(group_scaled[i + N_FUTURE - 1:i + N_FUTURE, len(FEATURES)])

# --- Proses Data Uji ---
print("Memproses Data Uji...")
# Simpan indeks asli dari data uji untuk merekonstruksi hasil nanti
test_indices = []
for meter_id, group in test_df.groupby('meter_id'):
    if meter_id in scalers:
        # 1. Scaling: Gunakan scaler dari data latih (HANYA transform)
        scaler = scalers[meter_id]
        group_scaled = scaler.transform(group[FEATURES + [TARGET]])

        # 2. Buat Sekuens
        for i in range(N_PAST, len(group_scaled) - N_FUTURE + 1):
            X_test.append(group_scaled[i - N_PAST:i, 0:len(FEATURES)])
            y_test.append(group_scaled[i + N_FUTURE - 1:i + N_FUTURE, len(FEATURES)])
            # Simpan indeks baris asli dari DataFrame test_df
            test_indices.append(group.index[i + N_FUTURE - 1])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

print(f"\nBentuk data latih (X, y): {X_train.shape}, {y_train.shape}")
print(f"Bentuk data uji (X, y): {X_test.shape}, {y_test.shape}")



Memproses Data Latih...
Memproses Data Uji...

Bentuk data latih (X, y): (282555, 24, 14), (282555, 1)
Bentuk data uji (X, y): (47541, 24, 14), (47541, 1)


### Tahap 3: Pelatihan Model LSTM

In [ ]:
# --- Bangun Arsitektur Model ---
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dense(y_train.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

# --- Latih Model ---
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 24, 64)         │        20,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,673 (127.63 KB)

 Trainable params: 32,673 (127.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
 719/7947 ━━━━━━━━━━━━━━━━━━━━ 2:45 23ms/step - loss: 0.0197

### Tahap 4: Prediksi dan Rekonstruksi Hasil (Paling Kritis)

In [ ]:
# ==============================================================================
# --- Tahap 4: Prediksi dan Rekonstruksi Hasil (Metodologi Benar) ---
# (Kode ini identik, hanya menggunakan model_1_layer untuk prediksi)
# ==============================================================================

# --- Lakukan Prediksi ---
predictions_scaled = model_1_layer.predict(X_test)

# --- Buat DataFrame Hasil yang Solid ---
df_hasil = test_df.loc[test_indices].copy()
y_pred_inversed = np.array([])
y_test_inversed = np.array([])

# --- Lakukan inverse transform per gedung ---
for meter_id, group in df_hasil.groupby('meter_id'):
    if meter_id in scalers:
        group_indices = group.index
        posisi = [test_indices.index(i) for i in group_indices]
        preds_scaled_group = predictions_scaled[posisi]
        test_scaled_group = y_test[posisi]
        dummy_pred = np.zeros((len(preds_scaled_group), len(FEATURES) + 1)); dummy_pred[:, -1] = preds_scaled_group.ravel()
        dummy_test = np.zeros((len(test_scaled_group), len(FEATURES) + 1)); dummy_test[:, -1] = test_scaled_group.ravel()
        inversed_preds = scalers[meter_id].inverse_transform(dummy_pred)[:, -1]
        inversed_tests = scalers[meter_id].inverse_transform(dummy_test)[:, -1]
        y_pred_inversed = np.append(y_pred_inversed, inversed_preds)
        y_test_inversed = np.append(y_test_inversed, inversed_tests)

# --- Tambahkan kolom hasil ke DataFrame ---
df_hasil['prediksi_lstm'] = y_pred_inversed
df_hasil.rename(columns={TARGET: 'target_aktual'}, inplace=True)

# --- Evaluasi Akhir (DIPERBARUI DENGAN METRIK TAMBAHAN DAN PENJELASAN) ---
y_true = df_hasil['target_aktual']
y_pred = df_hasil['prediksi_lstm']

# Definisikan fungsi untuk MAPE dan sMAPE untuk menghindari pembagian dengan nol
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Menambahkan epsilon kecil untuk menghindari pembagian dengan nol
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Menambahkan epsilon kecil untuk menghindari pembagian dengan nol di kedua sisi
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8)) * 100

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)

print(f"\n--- Evaluasi Final yang Konsisten untuk LSTM 1-Lapis ---")
print("\n")
print(f"Mean Absolute Error (MAE):       {mae:.4f}")
print("--> Penjelasan: Rata-rata selisih absolut antara prediksi dan nilai aktual. Satuannya sama dengan target (kWh). Semakin kecil, semakin baik.")
print("\n")
print(f"Root Mean Squared Error (RMSE):  {rmse:.4f}")
print("--> Penjelasan: Mirip MAE, tapi lebih menghukum kesalahan besar karena dikuadratkan. Satuannya juga kWh. Semakin kecil, semakin baik.")
print("\n")
print(f"R-squared (R²):                  {r2:.4f}")
print("--> Penjelasan: Seberapa baik model menjelaskan variasi data. Nilai 1 berarti prediksi sempurna. Semakin mendekati 1, semakin baik.")
print("\n")
print(f"Symmetric MAPE (sMAPE):          {smape:.2f}%")
print("--> Penjelasan: Versi perbaikan dari MAPE, lebih stabil jika ada nilai aktual mendekati nol. Memberikan error dalam bentuk persentase. Semakin kecil, semakin baik.")
print("\n")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print("--> Peringatan: Nilai MAPE sangat besar! Ini terjadi karena beberapa nilai aktual sangat mendekati nol. Gunakan sMAPE sebagai alternatif yang lebih stabil.")


# --- Simpan Hasil dengan Nama Berbeda ---
output_filename = 'lstm_1_layer_results.parquet'
df_hasil[['timestamp', 'meter_id', 'target_aktual', 'prediksi_lstm']].to_parquet(
    os.path.join(PATH_PREDICTIONS, output_filename), index=False
)
print(f"\nDataFrame hasil LSTM (1 Lapis) yang sudah sejajar berhasil disimpan ke:\n{os.path.join(PATH_PREDICTIONS, output_filename)}")



1486/1486 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step

Evaluasi Final yang Konsisten:
MAE: 2.4840
R-squared (R²): 0.9579

DataFrame hasil LSTM yang sudah sejajar berhasil disimpan.
